In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [2]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [3]:
colnames(data_source)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [4]:
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
returns = merge(returns, log1p.(data_source[:T_Bill])./4)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
term = data_source[:Yeld_10Y] ./4 .- data_source[:T_Bill] ./4
dy = data_source[:Dividend_Yeld] #log.(data_source[:Dividend_Yeld])
data = merge(real_returns, dy, term, cpi)

291×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-03-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1952-03-01 │      1.98335e-5 │     0.0107337 │  0.00772445 │     0.0602359 │ ⋯
│ 1952-06-01 │       0.0581411 │    0.00263202 │ -0.00708528 │      0.057815 │ ⋯
│ 1952-09-01 │      -0.0185027 │    -0.0173351 │  0.00423886 │     0.0592197 │ ⋯
│ 1952-12-01 │       0.0932838 │     0.0220972 │   0.0089235 │     0.0538579 │ ⋯
│ 1953-03-01 │      -0.0437093 │   -0.00972202 │  0.00497517 │     0.0571967 │ ⋯
│ 1953-06-01 │      -0.0100139 │    -0.0219485 │ -0.00227055 │     0.0584603 │ ⋯
│ 1953-09-01 │     -0.00574595 │     0.0298874 │ -0.00299956 │     0.0596579 │ ⋯
│ 1953-12-01 │        0.078226 │     0.0248173 │  0.00767892 │     0.0572141 │ ⋯
│     ⋮      │        ⋮        │ 

In [19]:
CSV.write("usa_data_transformed.csv", data)

"usa_data_transformed.csv"

In [20]:
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [7]:
returns_summarystats(to(from(returns, start),end_d), 4)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.144099,0.782737,0.0963943,0.261158,-1.01471,-0.845921,-0.0423317,0.0953652,1.77521,0.668946,0.142769
Price_10Y,0.0832445,0.569087,0.044665,0.156403,-0.289949,0.371142,-0.0740101,0.0442422,0.221941,0.536551,0.0280389
T_Bill,0.0101058,0.0582689,0.0227848,0.045069,9.9995e-5,0.292382,0.00149888,0.968883,-1.50635,2.25464,0.0172994


In [8]:
returns_summarystats(s_data, 1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.0717569,0.193813,0.0178775,0.0610642,-0.238141,-0.699168,-0.0145302,0.0849045,1.2878,0.24914,0.0283261
Price_10Y_CPI,0.0448954,0.167664,0.0049452,0.0347394,-0.0960374,0.432308,-0.0258376,0.0666902,0.789709,0.110149,0.00203425
T_Bill_CPI,0.00893438,0.025467,-0.000524846,0.00528326,-0.026825,-0.318287,-0.00503521,0.302165,0.492507,-0.0587446,0.000505661
Dividend_Yeld,0.00435984,0.0323637,0.0189393,0.0207695,0.0112582,0.765491,0.0164025,0.918403,0.717749,4.34404,0.0185857
Yeld_10Y_T_Bill,0.00293746,0.0095,0.00388508,0.006225,-0.0034,-0.236784,0.0017125,0.874579,-0.53084,1.3226,0.00395
CPI,0.00774615,0.0279226,0.00622104,0.00985738,-0.025392,-0.34528,0.00268687,0.0727647,2.47992,0.803114,0.00575376


In [9]:
returns_summarystats(s_data, 4)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.143514,0.775253,0.0715101,0.244257,-0.952565,-0.699168,-0.0581207,0.0849045,1.2878,0.498281,0.113304
Price_10Y_CPI,0.0897907,0.670655,0.0197808,0.138958,-0.38415,0.432308,-0.10335,0.0666902,0.789709,0.220299,0.00813698
T_Bill_CPI,0.0178688,0.101868,-0.00209938,0.021133,-0.1073,-0.318287,-0.0201408,0.302165,0.492507,-0.117489,0.00202265
Dividend_Yeld,0.00871968,0.129455,0.0757573,0.0830779,0.0450328,0.765491,0.06561,0.918403,0.717749,8.68808,0.0743426
Yeld_10Y_T_Bill,0.00587492,0.038,0.0155403,0.0249,-0.0136,-0.236784,0.00685,0.874579,-0.53084,2.6452,0.0158
CPI,0.0154923,0.11169,0.0248842,0.0394295,-0.101568,-0.34528,0.0107475,0.0727647,2.47992,1.60623,0.0230151


In [10]:
vmodel = VARModel(s_data)
model_summary(vmodel)

,const,Total_Stock_CPI,Price_10Y_CPI,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,CPI,std
Total_Stock_CPI,-0.0295,0.1152,0.0788,-0.8405,4.0279,-5.652,-1.5289,0.07
Price_10Y_CPI,-0.0077,-0.1285,-0.0358,2.8038,-0.488,3.675,1.779,0.0427
T_Bill_CPI,-0.0122,-0.0249,-0.0146,1.0719,0.3155,0.119,1.0222,0.0073
Dividend_Yeld,0.0012,-0.003,-0.0016,0.002,0.9069,0.0919,0.0168,0.0015
Yeld_10Y_T_Bill,0.0003,0.0017,0.0066,-0.0515,0.0091,0.887,-0.0329,0.0012
CPI,0.0118,0.0262,0.0089,-0.0735,-0.3003,-0.0871,-0.0139,0.0075


,Total_Stock_CPI,Price_10Y_CPI,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,CPI
Total_Stock_CPI,1.0,-0.16,0.03,-0.91,0.02,-0.01
Price_10Y_CPI,-0.16,1.0,0.44,0.23,-0.58,-0.49
T_Bill_CPI,0.03,0.44,1.0,0.16,-0.14,-0.99
Dividend_Yeld,-0.91,0.23,0.16,1.0,-0.07,-0.17
Yeld_10Y_T_Bill,0.02,-0.58,-0.14,-0.07,1.0,0.07
CPI,-0.01,-0.49,-0.99,-0.17,0.07,1.0


In [11]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

"Total_Stock_CPI coefficients"

Quantiles
       parameters       2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol    Float64   Float64   Float64   Float64   Float64 

            const    -0.0976   -0.0523   -0.0286   -0.0055    0.0389
  Total_Stock_CPI    -0.0743    0.0510    0.1142    0.1790    0.3049
    Price_10Y_CPI    -0.2709   -0.0417    0.0794    0.1983    0.4245
       T_Bill_CPI    -3.9032   -1.8692   -0.8395    0.1904    2.1020
    Dividend_Yeld     0.7049    2.8687    4.0281    5.1421    7.3266
  Yeld_10Y_T_Bill   -11.4476   -7.7050   -5.6756   -3.7405   -0.0394
              CPI    -5.0411   -2.7071   -1.5349   -0.3587    1.9438


"Price_10Y_CPI coefficients"

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const   -0.0486   -0.0218   -0.0074    0.0069    0.0338
  Total_Stock_CPI   -0.2389   -0.1656   -0.1275   -0.0888   -0.0128
    Price_10Y_CPI   -0.2442   -0.1056   -0.0345    0.0394    0.1750
       T_Bill_CPI    0.9382    2.1575    2.7923    3.4362    4.6228
    Dividend_Yeld   -2.5300   -1.1914   -0.5114    0.2004    1.4955
  Yeld_10Y_T_Bill    0.1186    2.4934    3.7010    4.8989    7.2293
              CPI   -0.3173    1.0189    1.7551    2.4996    3.9343


"T_Bill_CPI coefficients"

...

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const   -0.0009   -0.0001    0.0003    0.0007    0.0015
  Total_Stock_CPI   -0.0015    0.0006    0.0017    0.0028    0.0049
    Price_10Y_CPI    0.0004    0.0045    0.0065    0.0087    0.0128
       T_Bill_CPI   -0.1052   -0.0696   -0.0514   -0.0335    0.0025
    Dividend_Yeld   -0.0493   -0.0102    0.0093    0.0297    0.0684
  Yeld_10Y_T_Bill    0.7861    0.8519    0.8865    0.9214    0.9891
              CPI   -0.0939   -0.0535   -0.0324   -0.0119    0.0291


"CPI coefficients"

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const    0.0044    0.0092    0.0117    0.0143    0.0189
  Total_Stock_CPI    0.0062    0.0193    0.0261    0.0330    0.0459
    Price_10Y_CPI   -0.0290   -0.0041    0.0084    0.0215    0.0465
       T_Bill_CPI   -0.3957   -0.1828   -0.0744    0.0353    0.2559
    Dividend_Yeld   -0.6447   -0.4173   -0.2959   -0.1749    0.0602
  Yeld_10Y_T_Bill   -0.7188   -0.3051   -0.0835    0.1237    0.5185
              CPI   -0.3877   -0.1368   -0.0142    0.1167    0.3627


"correlation matrix"

Quantiles
                       parameters      2.5%     25.0%     50.0%     75.0%      ⋯
                           Symbol   Float64   Float64   Float64   Float64   Fl ⋯

  Total_Stock_CPI_Total_Stock_CPI    0.0632    0.0686    0.0718    0.0751    0 ⋯
    Total_Stock_CPI_Price_10Y_CPI   -0.3320   -0.2165   -0.1571   -0.0930    0 ⋯
       Total_Stock_CPI_T_Bill_CPI   -0.1547   -0.0335    0.0297    0.0940    0 ⋯
    Total_Stock_CPI_Dividend_Yeld   -0.9389   -0.9223   -0.9123   -0.9012   -0 ⋯
  Total_Stock_CPI_Yeld_10Y_T_Bill   -0.1608   -0.0421    0.0215    0.0843    0 ⋯
              Total_Stock_CPI_CPI   -0.1927   -0.0724   -0.0071    0.0569    0 ⋯
    Price_10Y_CPI_Total_Stock_CPI   -0.3320   -0.2165   -0.1571   -0.0930    0 ⋯
      Price_10Y_CPI_Price_10Y_CPI    0.0387    0.0418    0.0438    0.0459    0 ⋯
         Price_10Y_CPI_T_Bill_CPI    0.2708    0.3810    0.4352    0.4858    0 ⋯
      Price_10Y_CPI_Dividend_Yeld    0.0452    0.1659    0.2277    0.2874    0 ⋯
    Price_10Y_CPI

In [12]:
vmodel.Y[end,:]

6-element Vector{Float64}:
  0.12159459085777868
  0.06515539231011935
  0.010346584402051042
  0.014635185560818083
 -0.0034000000000000002
  0.002421733143949112

In [26]:
names = colnames(s_data)
scenarios = simulate(vmodel,60)

for n in 1:6
    s_ann = annualise(scenarios[n,:,2:end],4)
    print_percentiles(s_ann,[0.05, 0.25, 0.5, 0.75, 0.95], string(names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0907568,-0.162607,-0.180619,-0.195834,-0.195645,-0.194995,-0.201459,-0.193707,-0.19039,-0.197146,-0.198638,-0.193441,-0.192305,-0.195222,-0.193263
0.25,0.0429251,-0.0234794,-0.0368475,-0.048417,-0.0528254,-0.0493528,-0.0513998,-0.0504775,-0.0441791,-0.0490657,-0.0509064,-0.0470626,-0.0441101,-0.0500946,-0.0490974
0.5,0.13368,0.0764165,0.0623955,0.0512985,0.0504521,0.0528459,0.0519416,0.0518604,0.0547197,0.0522281,0.0502539,0.0542597,0.057414,0.0547111,0.0558084
0.75,0.22404,0.175282,0.163129,0.156684,0.152862,0.152732,0.153116,0.155582,0.153789,0.1522,0.157842,0.153799,0.157928,0.157896,0.156626
0.95,0.351511,0.317455,0.309028,0.304551,0.300621,0.302182,0.299041,0.302749,0.301781,0.297316,0.299385,0.299553,0.303958,0.303281,0.302948


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.150246,-0.135772,-0.135551,-0.134853,-0.135123,-0.133735,-0.138865,-0.13602,-0.143327,-0.1391,-0.135868,-0.142902,-0.142583,-0.142183,-0.144067
0.25,-0.0739984,-0.0518995,-0.0502999,-0.0483856,-0.0486768,-0.0477694,-0.049977,-0.0495692,-0.0533516,-0.0519285,-0.0505732,-0.0545023,-0.0546613,-0.052847,-0.0540543
0.5,-0.0188795,0.00603969,0.0104752,0.0118604,0.0126678,0.0128663,0.0115408,0.0112661,0.0085328,0.00904584,0.00825549,0.0063176,0.00591283,0.00774073,0.00742377
0.75,0.0340825,0.0643176,0.0698389,0.0727574,0.0724743,0.0707854,0.0718177,0.0712787,0.0698473,0.0726816,0.0703535,0.0673455,0.0682769,0.0683295,0.0671537
0.95,0.11236,0.150249,0.154319,0.160652,0.160622,0.159771,0.157914,0.157966,0.15828,0.157632,0.15821,0.158965,0.158614,0.158407,0.154585


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0120286,-0.0156628,-0.0207667,-0.0249497,-0.0285421,-0.0305848,-0.0328486,-0.0354176,-0.0365764,-0.0379235,-0.0397017,-0.040161,-0.0413513,-0.0408047,-0.0417588
0.25,0.00272873,0.00145244,-0.00230436,-0.00517311,-0.00751882,-0.0094632,-0.0112565,-0.0129289,-0.0145671,-0.0155209,-0.0167773,-0.0170271,-0.0177005,-0.0181812,-0.0187286
0.5,0.0124707,0.0129148,0.0104048,0.00848617,0.00664958,0.00469024,0.00349298,0.00234179,0.000899798,-4.55104e-5,-0.000720662,-0.000884755,-0.00218926,-0.00256687,-0.00275545
0.75,0.022378,0.024645,0.0235317,0.0220069,0.021365,0.0197526,0.0189273,0.017869,0.0166739,0.0158838,0.0150376,0.0147,0.0139805,0.0134689,0.0136482
0.95,0.036587,0.0420399,0.0418514,0.0427922,0.0416936,0.0409681,0.0409162,0.0401361,0.0399656,0.0386697,0.0388803,0.0375311,0.0368664,0.0370706,0.0361115


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,0.0417424,0.0339279,0.0312381,0.0305186,0.0308668,0.031339,0.0322833,0.0332069,0.033626,0.0346464,0.034507,0.035688,0.0359716,0.0364996,0.0367901
0.25,0.048819,0.0451052,0.0439163,0.044358,0.0452897,0.0459986,0.0472339,0.0482411,0.0490196,0.0497933,0.050631,0.0512502,0.0520133,0.0526556,0.0531687
0.5,0.0540644,0.0526448,0.0527594,0.0539221,0.0553073,0.056606,0.0577902,0.0590438,0.0599812,0.0608913,0.0617703,0.0625767,0.0630441,0.0640279,0.0643713
0.75,0.0591513,0.0603002,0.0616172,0.0633483,0.0651911,0.0669213,0.068509,0.0697802,0.0706601,0.0716434,0.0729748,0.0736598,0.0743818,0.075009,0.0756367
0.95,0.0666508,0.0716736,0.0745908,0.0769861,0.0791164,0.0812555,0.0835783,0.0849339,0.0863768,0.0873777,0.0888867,0.0896274,0.0904625,0.0915942,0.0924997


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0180989,-0.0194135,-0.0191686,-0.0183111,-0.0176506,-0.0169242,-0.0160168,-0.0155222,-0.0147306,-0.0143279,-0.0139133,-0.0137049,-0.0130048,-0.0131663,-0.0123992
0.25,-0.0125222,-0.0111823,-0.00940138,-0.00806731,-0.00682473,-0.00534574,-0.00421955,-0.00370226,-0.002752,-0.00202344,-0.00147994,-0.00102085,-0.000437825,-5.67828e-5,0.000200575
0.5,-0.0087742,-0.00532871,-0.00284033,-0.000933878,0.000981689,0.00240258,0.00364841,0.00476944,0.00555003,0.00655472,0.00707429,0.0075679,0.00823662,0.00884475,0.00918958
0.75,-0.00495305,0.000419199,0.00383834,0.00644313,0.00863751,0.010391,0.0118327,0.0130387,0.0138953,0.0150689,0.0159132,0.0166687,0.0172604,0.0177774,0.0181241
0.95,0.000472644,0.0085994,0.0135443,0.0170163,0.0196491,0.0216467,0.0235914,0.0249225,0.02612,0.0274991,0.0282521,0.0293826,0.0298407,0.0304821,0.0307161


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,0.00990358,0.00592786,0.00491399,0.00484066,0.00356546,0.00346943,0.00238264,0.00266462,0.00229231,0.00224062,0.00174168,0.00154257,0.000966297,0.0012163,0.000548907
0.25,0.0252781,0.0217172,0.0211168,0.0202807,0.0200152,0.0197705,0.0187804,0.0181736,0.0189912,0.0178747,0.0178627,0.0176776,0.017393,0.0170298,0.0171366
0.5,0.0360597,0.0325478,0.0320888,0.0316957,0.0312077,0.0304874,0.0300365,0.0301781,0.0299717,0.0294647,0.0290341,0.0286988,0.0286534,0.0281751,0.0282519
0.75,0.0467572,0.0433573,0.0435606,0.0431512,0.0424639,0.0417555,0.0414721,0.0412194,0.0409629,0.0409642,0.0402818,0.039741,0.0402596,0.0393061,0.0394891
0.95,0.0620312,0.0594414,0.0589034,0.0590106,0.0583057,0.0580793,0.0575229,0.0569416,0.0564994,0.0567742,0.0561788,0.0561709,0.056842,0.0564747,0.0556277


In [14]:
scenarios_b = simulate(bmodel,60, 10)
for n in 1:6
    s_ann = annualise(scenarios_b[n,:,:],4)
    print_percentiles(s_ann,[0.02, 0.25, 0.5, 0.75, 0.98], string(names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.0187596,-0.489224,-0.585559,-0.627915,-0.653624,-0.675245,-0.697163,-0.706453,-0.718323,-0.731371,-0.747488,-0.765243,-0.787476,-0.796294,-0.807554
0.25,0.163851,-0.0714041,-0.109127,-0.123938,-0.136093,-0.140101,-0.145547,-0.148556,-0.149645,-0.148111,-0.151325,-0.153158,-0.154593,-0.156969,-0.154429
0.5,0.226089,0.0826575,0.069608,0.0635469,0.0626082,0.0608963,0.0600277,0.0580562,0.060589,0.0632015,0.0638813,0.062372,0.0619291,0.0618618,0.0637476
0.75,0.294943,0.243492,0.2514,0.254782,0.258398,0.263548,0.264433,0.267215,0.271637,0.275236,0.278242,0.281415,0.28489,0.285103,0.288405
0.98,0.484317,0.695536,0.751714,0.781807,0.802921,0.828942,0.848446,0.875276,0.895687,0.917731,0.944282,0.989613,1.01839,1.04389,1.07401


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.122557,-0.47995,-0.555899,-0.595112,-0.625889,-0.648215,-0.65813,-0.670925,-0.680427,-0.689429,-0.693579,-0.700746,-0.71416,-0.71515,-0.731262
0.25,0.00138367,-0.124839,-0.151509,-0.165497,-0.175345,-0.184506,-0.189072,-0.195536,-0.1982,-0.203497,-0.204631,-0.207802,-0.209012,-0.210964,-0.2154
0.5,0.0481946,0.00676334,0.0107151,0.0104821,0.010075,0.00861161,0.00697647,0.00708107,0.00386336,0.00373133,0.00340787,0.00328117,0.00127545,0.00332755,0.000409537
0.75,0.0924794,0.136081,0.171577,0.186077,0.194586,0.200395,0.204258,0.207758,0.210005,0.209067,0.210206,0.211992,0.214257,0.213378,0.216265
0.98,0.196697,0.47116,0.571558,0.617943,0.639253,0.657597,0.664404,0.671854,0.678673,0.688902,0.69275,0.696595,0.705027,0.709811,0.716955


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0340416,-0.127104,-0.175468,-0.207642,-0.238388,-0.264147,-0.286067,-0.304844,-0.324033,-0.339606,-0.356999,-0.37187,-0.389308,-0.407918,-0.42414
0.25,0.00256682,-0.0291984,-0.0454723,-0.0570557,-0.0654643,-0.0724696,-0.078113,-0.0824728,-0.0867631,-0.0894651,-0.0910365,-0.0940114,-0.0954906,-0.0966493,-0.0989067
0.5,0.0189685,0.0136725,0.0116371,0.00921372,0.00691987,0.00557503,0.004252,0.00359599,0.00298369,0.00299367,0.00235044,0.00258506,0.000968521,0.00111887,0.000923449
0.75,0.0354113,0.0567617,0.0679661,0.0749355,0.0801448,0.0838816,0.0875058,0.0900668,0.0920761,0.0941558,0.0960473,0.0983587,0.100247,0.101171,0.102526
0.98,0.0712445,0.15273,0.196683,0.226569,0.252156,0.274931,0.296505,0.315817,0.335703,0.358679,0.380551,0.404588,0.430536,0.455671,0.490088


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.0173166,-0.0294988,-0.0551872,-0.0748185,-0.0938196,-0.110057,-0.127657,-0.145326,-0.16355,-0.181878,-0.204596,-0.227553,-0.250773,-0.275455,-0.302562
0.25,0.0432445,0.0273974,0.0207103,0.016868,0.0139645,0.0117965,0.0101931,0.00830152,0.0069085,0.00671924,0.00581374,0.00455035,0.00338487,0.00234705,0.00141045
0.5,0.0554526,0.0531422,0.0531289,0.0538518,0.0547389,0.0559,0.0567541,0.0577564,0.0588517,0.0599183,0.0603947,0.0609548,0.0614437,0.0614037,0.0621119
0.75,0.0676266,0.0785988,0.0844747,0.0898023,0.094054,0.0978152,0.101269,0.104343,0.107169,0.109607,0.111314,0.113414,0.115268,0.116826,0.117786
0.98,0.093857,0.133136,0.154617,0.170435,0.185141,0.199511,0.212874,0.227206,0.240952,0.253994,0.269838,0.283784,0.29817,0.31313,0.328658


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0481636,-0.0859481,-0.10454,-0.119515,-0.133041,-0.147397,-0.162288,-0.176911,-0.193652,-0.21188,-0.231417,-0.251389,-0.273692,-0.297987,-0.326196
0.25,-0.022207,-0.0306791,-0.0336715,-0.0354882,-0.0368045,-0.0384644,-0.039922,-0.0409018,-0.042276,-0.0434765,-0.0445435,-0.0455727,-0.0462653,-0.0466289,-0.0475911
0.5,-0.0101633,-0.00603254,-0.00348047,-0.00135143,9.68247e-5,0.00142169,0.00283165,0.00360213,0.00421531,0.00468702,0.0052738,0.00575387,0.00614615,0.0058776,0.00611598
0.75,0.0017631,0.0182177,0.0259846,0.0316395,0.0361725,0.0397746,0.0435092,0.0458858,0.0480411,0.0502649,0.0520212,0.0536734,0.0550559,0.0560952,0.0570595
0.98,0.0271804,0.0706929,0.0915479,0.106534,0.120709,0.132933,0.144506,0.15708,0.170253,0.18176,0.193902,0.206751,0.21912,0.233022,0.246802


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.00519524,-0.0158206,-0.0278395,-0.0364759,-0.042936,-0.049729,-0.0562454,-0.0615214,-0.0676864,-0.0743842,-0.0806481,-0.086571,-0.0932025,-0.099853,-0.10841
0.25,0.0221843,0.0181253,0.0152045,0.0132538,0.0117719,0.0104088,0.0092774,0.00838089,0.00768503,0.00686847,0.00597882,0.00529767,0.00462418,0.00417536,0.00383885
0.5,0.030026,0.0318399,0.0314486,0.0310587,0.0308242,0.0305237,0.0303709,0.0300713,0.0297858,0.029717,0.0294957,0.0292189,0.0291297,0.0291122,0.0291537
0.75,0.0381008,0.0466071,0.0490323,0.0504894,0.0516086,0.0525111,0.0532924,0.053861,0.0542924,0.0546708,0.0553264,0.055767,0.0560676,0.0564863,0.0568409
0.98,0.0563328,0.0867722,0.0999601,0.108914,0.117327,0.125145,0.133021,0.141129,0.150198,0.15865,0.16954,0.181592,0.192092,0.204032,0.215863


In [15]:
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[4,20,40,60]] ./ [1 5 10 15]),[.05, .25, .5, .75, .95], string(names[n]))    
end

,1,2,3,4
0.05,-0.0864658,-0.00316099,0.0153218,0.0230165
0.25,0.0412724,0.0449237,0.0443563,0.0464532
0.5,0.143688,0.0780893,0.0650038,0.0625429
0.75,0.251772,0.113356,0.0864877,0.0787623
0.95,0.423213,0.166233,0.119897,0.103174


,1,2,3,4
0.05,-0.142835,-0.0313621,-0.0131912,-0.00898711
0.25,-0.0708897,-0.0106836,-0.0012392,0.00047649
0.5,-0.0188543,0.00457163,0.00748262,0.00719391
0.75,0.0353688,0.0200151,0.0162496,0.0140021
0.95,0.120672,0.0421668,0.0284066,0.0239796


,1,2,3,4
0.05,-0.0120856,-0.00758137,-0.0129995,-0.0153895
0.25,0.00254538,0.00288764,-0.0016193,-0.00421347
0.5,0.0127731,0.0100508,0.00610813,0.00347385
0.75,0.0227181,0.0171519,0.0136653,0.0112556
0.95,0.0371768,0.0274822,0.0251054,0.022509


In [16]:
returns_exp = expm1.(scenarios[[1,2,3],:,:])
for n in 1:3
    s_ann = annualise(returns_exp[n,:,2:end],4)
    print_percentiles(s_ann,[.1, .3, .5, .7, .9], string(names[n]))
end


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.0313432,-0.0995567,-0.11763,-0.125758,-0.128415,-0.12884,-0.135826,-0.134497,-0.126441,-0.132941,-0.132165,-0.129409,-0.126232,-0.127647,-0.12184
0.3,0.0686783,0.0116482,-0.00993958,-0.0164477,-0.0188337,-0.0198338,-0.0215716,-0.0228681,-0.0172416,-0.0203867,-0.0199886,-0.0169077,-0.01658,-0.0152056,-0.0119594
0.5,0.145047,0.0859915,0.0664817,0.0634159,0.0616897,0.0581413,0.05825,0.0567413,0.0662434,0.06354,0.0551248,0.0654154,0.0630721,0.0672361,0.0669548
0.7,0.219851,0.165544,0.146752,0.143772,0.141514,0.143265,0.139228,0.13699,0.144895,0.147066,0.136439,0.145949,0.142257,0.146652,0.147806
0.9,0.326657,0.279199,0.268784,0.264219,0.257623,0.258775,0.261383,0.253033,0.264054,0.263338,0.257217,0.266865,0.262311,0.264909,0.267485


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.120205,-0.0991146,-0.100068,-0.0984066,-0.100276,-0.101095,-0.100945,-0.103264,-0.104549,-0.101527,-0.102769,-0.104482,-0.10464,-0.106693,-0.105093
0.3,-0.0571694,-0.0346531,-0.0326926,-0.0318416,-0.031934,-0.0311864,-0.0339441,-0.0341428,-0.0359669,-0.0354013,-0.0355117,-0.0369888,-0.0366701,-0.0374927,-0.0387976
0.5,-0.0154722,0.0117135,0.014221,0.0142006,0.015108,0.0157935,0.0125692,0.0127927,0.0128633,0.0131108,0.0126274,0.0091445,0.012196,0.00952357,0.00814813
0.7,0.0249668,0.0564039,0.06033,0.0627887,0.0634993,0.0635031,0.0623252,0.0604178,0.0604193,0.06166,0.0605256,0.0573578,0.0588158,0.0567007,0.0562566
0.9,0.0878236,0.124697,0.129728,0.133903,0.13129,0.134121,0.134039,0.132879,0.130273,0.130805,0.130902,0.127455,0.129159,0.127165,0.128266


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.00648811,-0.0091976,-0.0146926,-0.0184664,-0.0210692,-0.023676,-0.0257764,-0.0270071,-0.0287491,-0.0296648,-0.0304743,-0.03118,-0.0317874,-0.0322616,-0.0326089
0.3,0.00498828,0.00406508,-1.66959e-5,-0.00314159,-0.00467819,-0.00702954,-0.00875151,-0.0101724,-0.0113377,-0.0120642,-0.0126499,-0.0134532,-0.0141143,-0.0141524,-0.014577
0.5,0.0128197,0.013126,0.0102416,0.00817945,0.00624449,0.00497814,0.0032156,0.00193102,0.00113089,0.000331268,-0.000710717,-0.00106302,-0.00138365,-0.00208907,-0.00190547
0.7,0.0204032,0.0220383,0.0205023,0.0187358,0.0173863,0.0163869,0.0151756,0.0141099,0.0134062,0.0125897,0.0113631,0.0110894,0.0108401,0.0101645,0.0103631
0.9,0.0311731,0.0353267,0.0352359,0.0341434,0.0338737,0.0328533,0.032337,0.0317011,0.0309293,0.0305325,0.0295552,0.0290425,0.0291191,0.0284719,0.027539


In [17]:
skewness(rand(Normal(0,1),10000))

-0.008391009480189442